In [1]:
import tensorflow as tf
import numpy as np
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda, Reshape
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
from keras.layers import Input, Embedding

import keras.backend as K


.....\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#******************** First part : preparing data *********************************
# load dataset, embeddings and define parameters 

Max_Seq_Length = 20             # try other values  
Max_Num_Words = 20000           # tokenization will be restricted to the top Max_Num_Words most common words in the dataset
Embedding_Dim = 50              # as we choose glove.6B.50d.txt 
Validation_Split = 0.2

In [3]:

# Build a dictionary "embeddings_index" " mapping each word to its embedding vector 
print('Creating ''embeddings_index'' dictionaty word:emdedding')
embeddings_index = {}
with open('data/glove.6B.50d.txt', 'r', encoding="utf8") as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


# Create list of lines of text from the original test file
texts=[]
with open('data/text_file.txt', 'r', encoding="utf8") as f:
                    for line in f:
                        i = line.find('\n')  # skip \n
                        if 0 < i:
                            line = line[:i]
                        texts.append(line)


# Vectorize the texts samples into a 2D integer tensor 
# sequences: a list of tokens 
# word_index: dictionary 
print("Creating dataset :")
t = Tokenizer(num_words=Max_Num_Words)
t.fit_on_texts(texts)
sequences = t.texts_to_sequences(texts)
word_index = t.word_index                                          # word_index will have the indexes for texts nad labels 
print('Found %s unique tokens.' % len(word_index))

data_all = pad_sequences(sequences, maxlen=Max_Seq_Length)         # Padding sequences into a same length: Max_Seq_Length
data = data_all[:np.shape(data_all)[0]//2][:]                      # Splitting data line into 2
labels = data_all[np.shape(data_all)[0]//2 :][:]                   # we need a text_file [All first sentences lines] then [All replies lines]


# Split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(Validation_Split * data.shape[0])


x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]
print("x_train shape",np.shape(x_train))
print("y_train shape",np.shape(y_train))
print("x_val shape" , np.shape(x_val))
print("y_val shape" , np.shape(y_val))


#-----------Creating the embedding matrix--------------------
# Embeddings matrix (50, 20000) (Embedding_Dim, num_words)

num_words = min(Max_Num_Words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, Embedding_Dim))
for word, i in word_index.items():       
    if i >= Max_Num_Words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
#-----------Creating the Keras embedding Layer----------------
# load pre-trained word embeddings into an Embedding layer
embedding_layer = Embedding(num_words,Embedding_Dim, weights=[embedding_matrix], input_length=Max_Seq_Length, trainable=False)



Creating embeddings_index dictionaty word:emdedding
Found 400000 word vectors.
Creating dataset :
Found 131 unique tokens.
x_train shape (8, 20)
y_train shape (8, 20)
x_val shape (2, 20)
y_val shape (2, 20)


In [4]:
#******************** 2nd part : helper functions ***************************


# softmax function

def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor
        axis: Integer, axis along which the softmax normalization is applied
    # Returns
        Tensor, output of softmax transformation
    # Raises
        ValueError: In case `dim(x) == 1`
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')


# Define layers as global variables 
repeat_tx = RepeatVector(Max_Seq_Length)
concatenate_2 = Concatenate(axis=2)
dense_10 = Dense(10, activation = "tanh")
dense_1 = Dense(1, activation = "relu") 
activate_1 = Activation(softmax, name='attention_weights') 
dot_1 = Dot(axes = 1)
Activation_ = Activation('softmax')

# one_step_attention function

def one_step_attention(a, s_prev):
    """
    One step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    s_prev = repeat_tx(s_prev)
    concat = concatenate_2([ a, s_prev])
    e = dense_10(concat)
    energies = dense_1(e)
    alphas = activate_1(energies)
    context = dot_1([alphas, a])
    
    return context


# Other functions

def value_to_indice(rand_value, vector):
    '''
    rand_value: a value in the vector type float64
    vector: numpay array
    return the indice of the first occurence of the value
    '''
    ind = np.nonzero(vector*1e16 == rand_value*1e16)
    
    indice_random_value = ind[0]
        
    return indice_random_value 


In [5]:
#******************** 3nd part : keras model **************************

# Defining layers as global variables - to have shared weights 
n_a = 32  # hidden state dimension for the post attention LSTM cell
n_s = 32  # hidden state dimension for the last LSTM cell 

reshap_ = Reshape((1, n_a,))  
post_attention_LSTM_cell = LSTM(n_s, return_state = True)  # Post attention-step LSTM cell
output_layer2 = Dense(num_words, activation=softmax) 
output_linearlayer = Dense(n_a, activation=None)           #  Linear linears 
output_linearlayer2 = Dense(n_a, activation=None)     
last_LSTM_cell = LSTM(n_a, return_state = True)            # Last LSTM cell 


In [6]:

def model(Tx, Ty, n_a, n_s, Embedding_Dim, num_words):
    """
    Model architecture: Bidirectional LSTM -> attention-step-> Post attention LSTM -> Generative LSTM
    Arguments:
    Tx -- length of the input sequence = Max_Seq_Length
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    Embedding_Dim : Word embeddings dimension
    num_words : total number of words

    Returns:
    model -- Keras model instance
    """
    

    #m=5 #number examples
    x_train = Input(shape=(Max_Seq_Length, ),  batch_shape=(None, Max_Seq_Length))  
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')   
    ss0 = Input(shape=(n_a,), name='ss0') 
    cc0 = Input(shape=( n_a,), name='cc0')  
    s = s0
    c = c0
    ss = ss0
    cc = cc0   
    
    Tx = Max_Seq_Length
    
    # outputs initiazing
    outputs2 = []
    outputs_one_hot =[]

    # Embedding layer
    x_embed = embedding_layer(x_train)

    # Bidirectional LSTM
    a = Bidirectional(LSTM(n_a, return_sequences=True))(x_embed)  
    
    # Iterate for Tx steps : attention step -> post attention LSTM 
    for t in range(Tx):
        context = one_step_attention( a, s)
        s, _, c = post_attention_LSTM_cell(context, initial_state = [s,c])  
    
    # Iterate for Ty steps : Generative last LSTM cell
    out20 = reshap_(output_linearlayer(s))
    
    for t in range(Ty):
        ss, _, cc = last_LSTM_cell(out20, initial_state = [ss,cc]) 
        out2 = output_layer2(ss)                                       
        out2 = Activation_(out2)
        out20 = reshap_(output_linearlayer2(out2))         
        outputs2.append(out2)

    print("outputs2 shape", np.shape(outputs2[0])) 
    
    # Generating the model
    model = Model(inputs= [x_train, s0,c0, ss0, cc0], outputs= outputs2)
    
    return model
 

In [7]:
model1 = model(20, 20, n_a, n_s, Embedding_Dim, num_words)


outputs2 shape (?, 132)


In [8]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  #

In [9]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 50)       6600        input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 32)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20, 64)       21248       embedding_1[0][0]                
__________________________________________________________________________________________________
repeat_vec

In [17]:
import pydot
import pydot_ng as pydot
from keras.utils.vis_utils import plot_model
from keras.utils import plot_model
plot_model(model1, to_file='model.png')

In [18]:
# Convert and reshape y_train to one-hot vectors 

y_train_onehot_ = to_categorical(y_train , num_words) 
y_train_oh_s = (np.array_split(y_train_onehot_, 20, axis=1))
for i in range(len(y_train_oh_s)):
    y_train_oh_s[i]=y_train_oh_s[i].squeeze()    

print("np.shape(y_train_oh_s)",np.shape( y_train_oh_s))

# Other entries
m=np.shape(x_train)[0]
s0 = np.zeros(( m, n_s))
c0 = np.zeros(( m, n_s))
ss0 = np.zeros(( m, n_a))
cc0 = np.zeros(( m, n_a))


np.shape(y_train_oh_s) (20, 8, 132)


In [21]:
model1.fit([x_train, s0,c0, ss0, cc0], list(y_train_oh_s), verbose=1, epochs = 1 , shuffle=False)

Epoch 1/1
8/8 [==============================] - 0s 14ms/step - loss: 87.5654 - activation_1_loss_1: 4.0348 - activation_1_loss_2: 4.0224 - activation_1_loss_3: 4.0218 - activation_1_loss_4: 4.0218 - activation_1_loss_5: 4.0218 - activation_1_loss_6: 4.0218 - activation_1_loss_7: 4.1466 - activation_1_loss_8: 4.1466 - activation_1_loss_9: 4.1466 - activation_1_loss_10: 4.1466 - activation_1_loss_11: 4.1466 - activation_1_loss_12: 4.5212 - activation_1_loss_13: 4.6460 - activation_1_loss_14: 4.6460 - activation_1_loss_15: 4.7709 - activation_1_loss_16: 4.7708 - activation_1_loss_17: 4.7709 - activation_1_loss_18: 4.7709 - activation_1_loss_19: 4.8957 - activation_1_loss_20: 4.8957 - activation_1_acc_1: 0.8750 - activation_1_acc_2: 0.8750 - activation_1_acc_3: 0.8750 - activation_1_acc_4: 0.8750 - activation_1_acc_5: 0.8750 - activation_1_acc_6: 0.8750 - activation_1_acc_7: 0.7500 - activation_1_acc_8: 0.7500 - activation_1_acc_9: 0.7500 - activation_1_acc_10: 0.7500 - activation_1_acc_1

In [22]:
# Convert and reshape y_val to one-hot vectors 

y_val_onehot_ = to_categorical(y_val , num_words)
y_val_oh_s = (np.array_split(y_val_onehot_, 20, axis=1))  
for i in range(len(y_val_oh_s)):
    y_val_oh_s[i]=y_val_oh_s[i].squeeze()
    
print("np.shape(y_val_oh_s)",np.shape( y_val_oh_s))

# Other entries
me=np.shape(x_val)[0]
s0e = np.zeros(( me, n_s))
c0e = np.zeros(( me, n_s))
ss0e = np.zeros(( me, n_a))
cc0e = np.zeros(( me, n_a))


np.shape(y_val_oh_s) (20, 2, 132)


In [23]:
loss = model1.evaluate([x_val, s0e,c0e, ss0e, cc0e], y_val_oh_s)

print("Test accuracy = ", loss)
#print("model.metrics_names", model1.metrics_names)

2/2 [==============================] - 2s 1s/step
Test accuracy =  [94.42579650878906, 4.403291702270508, 4.396644592285156, 4.396356582641602, 4.396323204040527, 4.396317958831787, 4.396317958831787, 4.396320343017578, 4.895713806152344, 4.895715713500977, 4.895713806152344, 4.895713806152344, 4.8957109451293945, 4.8957109451293945, 4.895705223083496, 4.895697116851807, 4.895716667175293, 4.895698547363281, 4.895698547363281, 4.89571475982666, 4.895713806152344, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [24]:
pred = model1.predict([x_val, s0e,c0e, ss0e, cc0e], batch_size=1, verbose=1, steps=None)

2/2 [==============================] - 2s 1s/step
